In [9]:
import numpy as np
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
# Initilzing the CNN
cnn = Sequential()

# Step 1 - create the convolution layer
cnn.add(Convolution2D(filters = 32, 
                      kernel_size = (5, 5), 
                      activation = 'relu', 
                      input_shape = (150, 150, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Convolution2D(32, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Convolution2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

 # Step 3 - Flatten to create the input vector
cnn.add(Flatten())

# Step 4 - add the fully connected layer
cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(units = 3, activation = 'softmax'))

In [3]:
# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [4]:
# Part 2 - fitting CNN to the images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)

In [17]:
training_set = train_datagen.flow_from_directory('./re-sorted_roof_images/',
                                                 target_size=(150, 150),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset='training')

validation_set = test_datagen.flow_from_directory('./re-sorted_roof_images/',
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   shuffle = False,
                                                   subset='validation')

Found 969 images belonging to 3 classes.
Found 241 images belonging to 3 classes.


In [8]:
cnn.fit_generator(training_set,
                        steps_per_epoch=len(training_set),
                        epochs=100,
                        validation_data=validation_set,
                        validation_steps = len(validation_set),
                        callbacks = [EarlyStopping(monitor='val_loss', 
                                     patience=30, 
                                     mode='auto', 
                                     restore_best_weights=True)]
                        )

Epoch 1/100
31/31 [==============================] - 23s 742ms/step - loss: 0.5346 - acc: 0.6927 - val_loss: 0.5233 - val_acc: 0.6736
Epoch 2/100
31/31 [==============================] - 22s 717ms/step - loss: 0.4964 - acc: 0.7045 - val_loss: 0.4950 - val_acc: 0.7165
Epoch 3/100
31/31 [==============================] - 22s 723ms/step - loss: 0.4937 - acc: 0.7342 - val_loss: 0.4972 - val_acc: 0.7344
Epoch 4/100
31/31 [==============================] - 22s 717ms/step - loss: 0.4767 - acc: 0.7421 - val_loss: 0.5195 - val_acc: 0.7261
Epoch 5/100
31/31 [==============================] - 22s 703ms/step - loss: 0.4719 - acc: 0.7396 - val_loss: 0.5220 - val_acc: 0.7165
Epoch 6/100
31/31 [==============================] - 22s 703ms/step - loss: 0.4618 - acc: 0.7525 - val_loss: 0.4486 - val_acc: 0.7552
Epoch 7/100
31/31 [==============================] - 22s 704ms/step - loss: 0.4515 - acc: 0.7524 - val_loss: 0.4462 - val_acc: 0.7441
Epoch 8/100
31/31 [==============================] - 22s 705ms

Epoch 62/100
31/31 [==============================] - 22s 722ms/step - loss: 0.2928 - acc: 0.8675 - val_loss: 0.3422 - val_acc: 0.8326
Epoch 63/100
31/31 [==============================] - 24s 771ms/step - loss: 0.2969 - acc: 0.8695 - val_loss: 0.3465 - val_acc: 0.8299
Epoch 64/100
31/31 [==============================] - 24s 767ms/step - loss: 0.2907 - acc: 0.8645 - val_loss: 0.3465 - val_acc: 0.8313
Epoch 65/100
31/31 [==============================] - 22s 717ms/step - loss: 0.3095 - acc: 0.8753 - val_loss: 0.3494 - val_acc: 0.8465
Epoch 66/100
31/31 [==============================] - 23s 741ms/step - loss: 0.3176 - acc: 0.8695 - val_loss: 0.3683 - val_acc: 0.8133
Epoch 67/100
31/31 [==============================] - 23s 758ms/step - loss: 0.2934 - acc: 0.8638 - val_loss: 0.3332 - val_acc: 0.8437
Epoch 68/100
31/31 [==============================] - 25s 801ms/step - loss: 0.2877 - acc: 0.8830 - val_loss: 0.3383 - val_acc: 0.8340
Epoch 69/100
31/31 [==============================] - 2

In [ ]:
#cnn.save('./pickled_models/categorical_model.h5')

In [18]:
y = validation_set.classes
y_pred = np.argmax(cnn.predict_generator(validation_set, len(validation_set)), axis=1)

In [21]:
print('Model Accuracy:')
print(round(accuracy_score(y, y_pred), 4))
print()
print('Confusion Matrix:')
print(confusion_matrix(validation_set.classes, y_pred))
print()
target_names = ['None', 'Good', 'Poor']
print('Classification Report:')
print(classification_report(validation_set.classes, y_pred, target_names=target_names))

Model Accuracy:
0.7261

Confusion Matrix:
[[  7   5   4]
 [  3 112  19]
 [  3  32  56]]

Classification Report:
             precision    recall  f1-score   support

       None       0.54      0.44      0.48        16
       Good       0.75      0.84      0.79       134
       Poor       0.71      0.62      0.66        91

avg / total       0.72      0.73      0.72       241

